In [ ]:
from IPython.display import YouTubeVideo

# Context

The data set comprises several sensor data collected from a permanent magnet synchronous motor (PMSM) deployed on a test bench. The PMSM represents a german OEM's prototype model. Test bench measurements were collected by the LEA department at Paderborn University.This data set is mildly anonymized.<br>

# Working of PMSM 


The below embedded vedio shows the working of the PMSM motor . Basically the motor consists of two parts the stator (meaning stationary) and rotor ( which rotates) . A alternating current is passed through the stator which generates a rotating magnetic field and a DC current is passed through the rotor which generates a stationary magnetic field . When the opposite poles of the two magnetic field unite the stator's rotating magnetic field drives the rotor's stationary magnetic field.<br> 







In [ ]:

YouTubeVideo('Vk2jDXxZIhs', width=800, height=500)

# How the data was collected 

All recordings are sampled at 2 Hz. The data set consists of multiple measurement sessions, which can be distinguished from each other by column "profile_id". A measurement session can be between one and six hours long.

The motor is excited by hand-designed driving cycles denoting a reference motor speed and a reference torque.
Currents in d/q-coordinates (columns "id" and iq") and voltages in d/q-coordinates (columns "ud" and "uq") are a result of a standard control strategy trying to follow the reference speed and torque.
Columns "motor_speed" and "torque" are the resulting quantities achieved by that strategy, derived from set currents and voltages.

Most driving cycles denote random walks in the speed-torque-plane in order to imitate real world driving cycles to a more accurate degree than constant excitations and ramp-ups and -downs would.

# Aim of this notebook 

My aim with this notebook was to predict the stator winding and rotor winding temperatures and select the best model out of various ones avaliable. 


In [ ]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline 

# Sklearn imports 
from sklearn.linear_model import Lasso,Ridge, LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn import model_selection
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn import neighbors

In [ ]:
data=pd.read_csv('/kaggle/input/electric-motor-temperature/pmsm_temperature_data.csv')

In [ ]:
data.head(20)


In [ ]:
data.shape

In [ ]:
data.isnull().sum()

# No Nulls values are detected 

In [ ]:
#data['profile_id'].value_counts().sort_index()

#Un commnet to see the values under each profile_id 

In [ ]:
plt.figure(figsize=(15,8));

sns.barplot(x=data.groupby('profile_id').agg('count').sort_values(by='pm').index,
            y=data.groupby('profile_id').agg('count').sort_values(by='pm')['pm'],
            order=data.groupby('profile_id').agg('count').sort_values(by='pm').index,
           orient='v',color='orange');

plt.title('Profile_id vs data points');

In the above graph we see that number of test points in each profile_id. This basically represents how long the test was run 

We see that max number of points are there in 65 ,6 and 20 

# Spliting into Train and Test set 

Given from desription we know that profile id 65 and 72 should be used in test set 

In [ ]:
X_test=data[(data['profile_id']==65) | (data['profile_id']==72)]
X_train=data[(data['profile_id']!=65) | (data['profile_id']!=72)]

In this dataset we need to predict the stator and rotor temperature given by 

__Target features__:<br>


stator_yoke<br>
stator_winding<br>
stator_tooth<br>
pm<br>


Rest all feature exluding torque as it is difficult to measure and profile_id as it is a label can be us for prediction 


In [ ]:
# Let see how the other features are colrelated with each other 

data.columns

In [ ]:
columns=['ambient', 'coolant', 'u_d', 'u_q', 'motor_speed', 'i_d',
       'i_q', 'pm', 'stator_yoke', 'stator_tooth', 'stator_winding','torque']

corr=data[columns].corr()

plt.figure(figsize=(12,10))
sns.heatmap(corr,annot=True);

## Points to note 

We see that the rotor temperature(pm) is corelated well with other features . Rotor temperature is also well corelated with 
stator temperarture so we can use predict rotor temperature first and then use that as an feature to predict stator temps

Also we see that there is a great corelation among various starter temperature so we can also decided if we want to predict one of the startor components only and assume the other around same temperature range . See the graph below 

Also we see that stator tooth is most of the time in between stator yoke and stator winding


Also torque is in well agreement with i_q

In [ ]:
#Change the profile ID number 

plt.figure(figsize=(20,5))
data[data['profile_id'] == 80]['stator_yoke'].plot(label = 'stator yoke')
data[data['profile_id'] == 80]['stator_tooth'].plot(label = 'stator tooth')
data[data['profile_id'] == 80]['stator_winding'].plot(label = 'stator winding')
plt.legend()

As part of dependent variables we have 4 temperature to predict .But out of the 4 temperatures required to predict 3 of stator usually follow the same profile and also are in the close vicinity of each other. Thus I feel if a model which can accurately model one of the three stator temperature should work for us. 


In [ ]:
data.describe()

From above we see that the min max and mean for all variables are with same value range hence I do not feel the need to 
normalize the data 


In [ ]:
X_train.head()

In [ ]:
X=X_train[['ambient','coolant','u_d','u_q','motor_speed','torque','i_d','i_q']]
y=X_train['pm']

y_test_rotor=X_test['pm']
X_test_rotor=X_test[['ambient','coolant','u_d','u_q','motor_speed','torque','i_d','i_q']]


#from sklearn.model_selection import train_test_split


#X_train,X_val,y_train,y_val=train_test_split(X,y,shuffle=True,random_state=7)

# Establishing a baseline using Linear Regression 

In [ ]:


lr=LinearRegression(normalize=False)

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

lr_scores=[]

lr_scores=cross_val_score(lr,X,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

    
print('RMSE for Linear Regression is {0:.3f} and variance of MSE is {1:.8f}'.format(-1*np.mean(lr_scores),np.var(lr_scores)))


1. ## Optimizing Ridge Regression 

In [ ]:

ridge=Ridge()
param_grid={
    'alpha':[0.001,0.01,1,5,10,20],
    'normalize' : [True,False],
    'fit_intercept':[True,False]
}

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

model=model_selection.RandomizedSearchCV(
                        estimator=ridge,
                        param_distributions=param_grid,
                        n_iter=20,
                        scoring='neg_root_mean_squared_error',
                        cv=kfold,
                        refit=True,
                        verbose=5,
                        n_jobs=-1
                        )

model.fit(X,y)

print('Best Scorer{}'.format(model.best_score_))

print('/n')

print('Best Parameters{}'.format(model.best_params_))



## Implementing the best fit Ridge model from above 

In [ ]:
# implementing Ridge Regression 

ridge=Ridge(alpha=5,fit_intercept=True,normalize=False)

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

ridge_scores=[]

ridge_scores=cross_val_score(ridge,X,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

    
print('RMSE for Ridge Regression is {0:.5f} and variance of MSE is {1:.8f}'.format(-1*np.mean(ridge_scores),np.var(ridge_scores)))

## Optimizing Lasso Regression 

In [ ]:
lasso=Lasso()

param_grid={
    'alpha':[0.001,0.01,1,5,10,20],
    'normalize' : [True,False],
    'fit_intercept':[True,False]
}

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

model=model_selection.RandomizedSearchCV(
                        estimator=lasso,
                        param_distributions=param_grid,
                        n_iter=20,
                        scoring='neg_mean_squared_error',
                        cv=kfold,
                        refit=True,
                        verbose=5,
                        n_jobs=-1
                        )

model.fit(X,y)

print('Best Scorer{}'.format(model.best_score_))

print('/n')

print('Best Parameters{}'.format(model.best_params_))



## Implementing the Best Lasso Regression

In [ ]:
 

lasso=Lasso(alpha=0.001,fit_intercept=True,normalize=False)

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

lasso_scores=[]

lasso_scores=cross_val_score(lasso,X,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

    
print('RMSE for Lasso Regression is {0:.5f} and variance of MSE is {1:.8f}'.format(-1*np.mean(lasso_scores),np.var(lasso_scores)))

## Most Linear Regression model gives us RMSE value close by so this can serve as good baseline model 

# Linear model with polynomial features 

I feel polynomial features should be good as this will create additional features . Also limitting the degree of polynomial to 2 as since usually Power generated within an electrical component has an effect on the temperature of the component and power is usually a product of 2 terms. 


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

lasso_pipeline=Pipeline([
    ('poly',PolynomialFeatures(2)),
    ('scalar',StandardScaler())
])



X_poly=lasso_pipeline.fit_transform(X)

# implementing Lasso Regression 

lasso=Lasso(alpha=0.001,fit_intercept=True,normalize=False)

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

lasso_scores=[]

lasso_scores=cross_val_score(lasso,X_poly,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

    
print('RMSE for Lasso Regression is {0:.5f} and variance of MSE is {1:.8f}'.format(-1*np.mean(lasso_scores),np.var(lasso_scores)))

## KNN Regressor 

In [ ]:
from sklearn import neighbors

knn=neighbors.KNeighborsRegressor()

param_grid={
    'n_neighbors':np.arange(1,10),
    'weights':['uniform', 'distance']   
    
}

kfold=model_selection.KFold(n_splits=5,shuffle=True,random_state=101)

model=model_selection.GridSearchCV(
                        estimator=knn,
                        param_grid=param_grid,
                        scoring='neg_root_mean_squared_error',
                        cv=kfold,
                        refit=True,
                        verbose=5,
                        n_jobs=-1
                        )
model.fit(X,y)

print('Best Scorer{}'.format(model.best_score_))

print('/n')

print('Best Parameters{}'.format(model.best_params_))


# Implementing the best KNN model 

In [ ]:
knn=neighbors.KNeighborsRegressor(n_neighbors=3,weights='distance')

kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

knn_scores=[]

knn_scores=cross_val_score(knn,X,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

    
print('RMSE for KNN Regression is {0:.5f} and variance of MSE is {1:.8f}'.format(-1*np.mean(knn_scores),np.var(knn_scores)))

In [ ]:
# As the KNN is so far the best one using KNN with sklearn BaggingClassifier class 

In [ ]:
#from sklearn.ensemble import BaggingRegressor
#from sklearn import neighbors

#knn_bag=BaggingRegressor(neighbors.KNeighborsRegressor(n_neighbors=3),
                          #max_samples=1.0, 
                          #max_features=1.0,
                         # oob_score=True,
                          #bootstrap=True)

#param_dist={
#    'n_estimators':np.arange(10,120,10)
#}

#model=model_selection.GridSearchCV(
#                        estimator=knn_bag,
 #                       param_grid=param_dist,
#            
 #                       scoring='neg_mean_squared_error',
#                        cv=5,
  #                      refit=True,
   #                     verbose=5,
     #                   n_jobs=-1
    #                    )


#model.fit(X,y)

#print('Best Scorer{}'.format(model.best_score_))

#print('/n')

#print('Best Parameters{}'.format(model.best_params_))

In [ ]:
# Random forest Regressor 

#from sklearn import ensemble

#random_forest=ensemble.RandomForestRegressor(criterion='mse',random_state=101,n_jobs=-1)

#param_distribution={
#    'n_estimators':np.arange(100,500,50),
#    'max_depth':[2,4,6,8,'None'], 
#    'min_samples_split':np.arange(2,20,2), 
#    'warm_start':[False,True]
#}


#kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

#model=model_selection.RandomizedSearchCV(
 #                       estimator=random_forest,
 #                       param_distributions=param_distribution,
 #                       n_iter=80,
 #                       scoring='neg_mean_squared_error',
 #                       cv=kfold,
                       # refit=True,
                       # verbose=5,
                       # n_jobs=-1
                       # )

#model.fit(X,y)

#print('Best Scorer{}'.format(model.best_score_))

#print('/n')

#print('Best Parameters{}'.format(model.best_params_))

In [ ]:
# Working on XGboost 
import xgboost as xgb 

data_dmatrix = xgb.DMatrix(data=X,label=y)

params = {"objective":"reg:squarederror",'colsample_bytree': 0.3,'learning_rate': 0.1,
                'max_depth': 5, 'alpha': 10}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=3,
                    num_boost_round=1500,early_stopping_rounds=10,metrics="rmse", as_pandas=True, seed=123)

cv_results.tail()

In [ ]:
# Implementing pipeline 
#from sklearn.pipeline import Pipeline
#from sklearn.ensemble import VotingRegressor
#import xgboost as xgb 


#votingregressor = VotingRegressor([('lasso',Lasso(alpha=0.001,fit_intercept=True,normalize=False)), 
  #                    ('knn', neighbors.KNeighborsRegressor(n_neighbors=3,weights='distance')),
  #                    ('xgb',xgb.XGBRegressor(objective='reg:squarederror',colsample_bytree=0.3,learning_rate=0.1,max_depth=5,alpha=10,n_rounds=1500))
    #                  ])


#vot_pipeline=Pipeline([
  #  ('poly',PolynomialFeatures(2)),
  #  ('scalar',StandardScaler()),
  #  ('voting',votingregressor)
#])

#kfold=model_selection.KFold(n_splits=10,shuffle=True,random_state=101)

#vot_scores=[]

#vot_scores=cross_val_score(vot_pipeline,X,y,cv=kfold,n_jobs=-1,scoring='neg_root_mean_squared_error')

#-1*np.mean(vot_scores)

# Final Working Model for Predicting for Predicting Rotor Temperature 

In [ ]:
KNN_rotor=neighbors.KNeighborsRegressor(n_neighbors=3,weights='distance')


KNN_rotor.fit(X,y)

y_rotor_predict=KNN_rotor.predict(X_test_rotor)

print("The RMSE for rotor temperature is {}".format(metrics.mean_squared_error(y_test_rotor,y_rotor_predict)))


# Implementing Model for Stator tooth 
the other stators temperature should be around this ballpark temperature


In [ ]:
X_stator_train=X_train[['ambient','coolant','u_d','u_q','motor_speed','torque','i_d','i_q']]
y_stator_train=X_train['stator_tooth']

y_stator_test=X_test['stator_tooth']
X_stator_test=X_test[['ambient','coolant','u_d','u_q','motor_speed','torque','i_d','i_q']]

In [ ]:
from sklearn import neighbors

knn=neighbors.KNeighborsRegressor()

param_grid={
    'n_neighbors':np.arange(1,15),
    'weights':['uniform', 'distance']   
    
}

kfold=model_selection.KFold(n_splits=5,shuffle=True,random_state=101)

stator_model=model_selection.GridSearchCV(
                        estimator=knn,
                        param_grid=param_grid,
                        scoring='neg_root_mean_squared_error',
                        cv=kfold,
                        refit=True,
                        verbose=5,
                        n_jobs=-1
                        )
stator_model.fit(X_stator_train,y_stator_train)

print('Best Scorer{}'.format(stator_model.best_score_))

print('/n')

print('Best Parameters{}'.format(stator_model.best_params_))

In [ ]:
KNN_stator=neighbors.KNeighborsRegressor(n_neighbors=3,weights='distance')


KNN_stator.fit(X_stator_train,y_stator_train)

y_stator_predict=KNN_stator.predict(X_stator_test)

print("The RMSE for rotor temperature is {}".format(metrics.mean_squared_error(y_stator_test,y_stator_predict)))
